# Eseguire uno script di training come processo di comando

È possibile usare Python SDK per Azure Machine Learning per inviare script come processi di comando. Usando i processi, è possibile tenere traccia facilmente dei parametri di input e degli output durante il training di un modello di Machine Learning.

## Prima di iniziare

È necessaria la versione più recente del pacchetto **azureml-ai-ml** per eseguire il codice in questo notebook. Eseguire la cella seguente per verificare che sia installata.

> **Nota**:
> Se il pacchetto **azure-ai-ml** non è installato, eseguirlo `pip install azure-ai-ml` per installarlo.

In [ ]:
## Connettersi all'area di lavoro

Con i pacchetti SDK necessari installati, è ora possibile connettersi all'area di lavoro.

Per connettersi a un'area di lavoro, sono necessari parametri di identificatore: ID sottoscrizione, nome del gruppo di risorse e nome dell'area di lavoro. Il nome del gruppo di risorse e il nome dell'area di lavoro sono già compilati. È necessario solo l'ID sottoscrizione per completare il comando.

Per trovare i parametri necessari, fare clic sulla sottoscrizione e sul nome dell'area di lavoro nella parte superiore destra di Studio. Un riquadro verrà aperto a destra.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiare l'ID sottoscrizione e sostituire **YOUR-SUBSCRIPTION-ID** con il valore copiato. </p>

## Rilevamento personalizzato con MLflow

Quando si esegue uno script come processo, è possibile usare MLflow nello script di training per tenere traccia del modello. MLflow consente di tenere traccia di tutti i parametri, le metriche o gli artefatti personalizzati da archiviare con l'output del processo.

Eseguire le celle seguenti per creare lo script **di train-model-mlflow.py** nella cartella **src** . Lo script esegue il training di un modello di classificazione usando il file **diabetes.csv** nella stessa cartella, che viene passato come argomento. 

Esaminare il codice seguente per trovare che lo script importerà `mlflow` e logrà:

- Frequenza di regolarizzazione come **parametro**. 
- Accuratezza e AUC come **metriche**.
- Curva ROC tracciata come **artefatto**.

In [ ]:
È ora possibile inviare lo script come processo di comando.

Eseguire la cella seguente per eseguire il training del modello. 

In [ ]:
In Studio passare al processo **diabete-training-mlflow** per esplorare la panoramica del processo di comando eseguito:

- Trovare i parametri registrati nella scheda **Panoramica** , in **Params**.
- Trovare le metriche registrate nella scheda **Metriche** .
- Trovare gli artefatti registrati nella scheda **Immagini** (in particolare per le immagini) e nella scheda **Output e log** (tutti i file).

## Assegnazione automatica dell'assegnazione automatica con MLflow

Anziché usare la registrazione personalizzata, MLflow può anche registrare automaticamente tutti i parametri, le metriche e gli artefatti. L'assegnazione automatica con MLflow richiede solo una riga di codice.

Eseguire la cella seguente per creare lo script **train-model-autolog.py** nella cartella **src** . Lo script esegue il training di un modello di classificazione usando il file **diabetes.csv** nella stessa cartella, che viene passato come argomento. 

Esaminare il codice seguente per trovare che lo script importerà `mlflow` e abilita l'assegnazione automatica dei tag con la riga: 

`mlflow.autolog()`

In [ ]:
È ora possibile inviare lo script come processo di comando.

Eseguire la cella seguente per eseguire il training del modello. 

In [ ]:
In Studio passare al processo **diabete-training-autolog** per esplorare la panoramica del processo di comando eseguito:

- Trovare i parametri registrati nella scheda **Panoramica** , in **Params**.
- Trovare le metriche registrate nella scheda **Metriche** .
- Trovare gli artefatti registrati nella scheda **Immagini** (in particolare per le immagini) e nella scheda **Output e log** (tutti i file, inclusi i file di modello).

## Usare MLflow per visualizzare e cercare esperimenti

Azure Machine Learning Studio è un'interfaccia utente facile da usare per visualizzare e confrontare le esecuzioni dei processi. In alternativa, è possibile usare MLflow per visualizzare i processi di esperimento. 

Per elencare i processi nell'area di lavoro, usare il comando seguente per elencare gli esperimenti nell'area di lavoro:


In [ ]:
Per recuperare un esperimento specifico, è possibile ottenerlo in base al nome:

Usando un nome di esperimento, è possibile recuperare tutti i processi dell'esperimento:

Per confrontare più facilmente le esecuzioni e gli output dei processi, è possibile configurare la ricerca per ordinare i risultati. Ad esempio, la cella seguente ordina i risultati in `start_time`base a e mostra solo un massimo di `2` risultati: 

In [ ]:
È anche possibile creare una query per filtrare le esecuzioni. Le stringhe di query di filtro vengono scritte con una versione semplificata della clausola SQL `WHERE` . 

Per filtrare, è possibile usare due classi di comparatori:

- Confronto numerico (metriche): =, !=, >, >=, < e <=.
- Confronto di stringhe (params, tag e attributi): = e !=.

Altre informazioni su [come tenere traccia degli esperimenti con MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

Now, you can submit the script as a command job.

Run the cell below to train the model. 

In [ ]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-autolog.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In the Studio, navigate to the **diabetes-train-autolog** job to explore the overview of the command job you ran:

- Find the logged parameters in the **Overview** tab, under **Params**.
- Find the logged metrics in the **Metrics** tab.
- Find the logged artifacts in the **Images** tab (specifically for images), and in the **Outputs + logs** tab (all files, including the model files).

## Use MLflow to view and search for experiments

The Azure Machine Learning Studio is an easy-to-use UI to view and compare job runs. Alternatively, you can use MLflow to view experiment jobs. 

To list the jobs in the workspace, use the following command to list the experiments in the workspace:


In [ ]:
import mlflow
experiments = mlflow.list_experiments()
for exp in experiments:
    print(exp.name)

To retrieve a specific experiment, you can get it by its name:

In [ ]:
experiment_name = "diabetes-training"
exp = mlflow.get_experiment_by_name(experiment_name)
print(exp)

Using an experiment name, you can retrieve all jobs of that experiment:

In [ ]:
mlflow.search_runs(exp.experiment_id)

To more easily compare job runs and outputs, you can configure the search to order the results. For example, the following cell orders the results by `start_time`, and only shows a maximum of `2` results: 

In [ ]:
mlflow.search_runs(exp.experiment_id, order_by=["start_time DESC"], max_results=2)

You can even create a query to filter the runs. Filter query strings are written with a simplified version of the SQL `WHERE` clause. 

To filter, you can use two classes of comparators:

- Numeric comparators (metrics): =, !=, >, >=, <, and <=.
- String comparators (params, tags, and attributes): = and !=.

Learn more about [how to track experiments with MLflow](https://learn.microsoft.com/azure/machine-learning/how-to-track-experiments-mlflow).

In [ ]:
query = "metrics.AUC > 0.8 and tags.model_type = 'LogisticRegression'"
mlflow.search_runs(exp.experiment_id, filter_string=query)